In [15]:
#libraries
from keras.datasets import cifar10
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import np_utils
import cv2
from tensorflow.keras import layers,Model
from tensorflow.keras.models import Sequential
from keras.layers import Flatten,Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.utils import to_categorical
IMAGE_SIZE=[32,32]
from sklearn.metrics import confusion_matrix



In [48]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

#separating data
#train
index1=np.where(y_train==0)
index2=np.where(y_train==1)
index3=np.where(y_train==9)
index=np.append(index1[0],index2[0])
index=np.append(index,index3[0])
X_train=X_train[index]
y_train=y_train[index]
for i in range(len(y_train)):
  if(y_train[i]==9):
    y_train[i]=2

#test
index1=np.where(y_test==0)
index2=np.where(y_test==1)
index3=np.where(y_test==9)
index=np.append(index1[0],index2[0])
index=np.append(index,index3[0])
X_test=X_test[index]
y_test=y_test[index]

for i in range(len(y_test)):
  if(y_test[i]==9):
    y_test[i]=2
    
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)




In [3]:
modelv =VGG16(input_shape=(32,32,3),weights='imagenet',include_top=False)


58892288/58889256 [==============================] - 0s 0us/step


In [4]:
for layer in modelv.layers:
	layer.trainable=False

In [5]:
x = Flatten()(modelv.output)
x = Dense(3, activation = 'softmax')(x)
model = Model(inputs = modelv.input, outputs = x)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
training_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=preprocess_input)

validation_datagen = ImageDataGenerator(rescale = 1./255, 
                                        preprocessing_function=preprocess_input)

In [9]:
training_datagen.fit(X_train)
validation_datagen.fit(X_test)

In [10]:
history=model.fit(training_datagen.flow(X_train, y_train, batch_size=200),
          steps_per_epoch=len(X_train) / 200, epochs=20,validation_data=(X_test,y_test),validation_steps = 3000)

Epoch 1/20
75/75 [==============================] - 27s 255ms/step - loss: 1.1569 - accuracy: 0.4343 - val_loss: 14.1327 - val_accuracy: 0.5743
Epoch 2/20
75/75 [==============================] - 19s 250ms/step - loss: 0.7797 - accuracy: 0.6828 - val_loss: 16.7314 - val_accuracy: 0.6060
Epoch 3/20
75/75 [==============================] - 19s 250ms/step - loss: 0.6893 - accuracy: 0.7212 - val_loss: 18.4770 - val_accuracy: 0.6227
Epoch 4/20
75/75 [==============================] - 19s 250ms/step - loss: 0.6514 - accuracy: 0.7348 - val_loss: 19.7124 - val_accuracy: 0.6287
Epoch 5/20
75/75 [==============================] - 19s 254ms/step - loss: 0.6235 - accuracy: 0.7433 - val_loss: 20.2508 - val_accuracy: 0.6333
Epoch 6/20
75/75 [==============================] - 19s 250ms/step - loss: 0.6026 - accuracy: 0.7557 - val_loss: 20.1927 - val_accuracy: 0.6450
Epoch 7/20
75/75 [==============================] - 19s 249ms/step - loss: 0.5930 - accuracy: 0.7578 - val_loss: 20.5185 - val_accuracy:

In [49]:
y_pred=model.predict(X_test)

In [50]:
y_pred=np.argmax(y_pred,axis=1)
y_test=np.argmax(y_test,axis=1)

In [51]:
print ('Training Accuracy = ' + str(history.history['accuracy'][-1]*100)+" %")
print ('Validation Accuracy = ' + str(history.history['val_accuracy'][-1]*100)+" %")
print('Validation loss = '+str(history.history['val_loss'][-1]))


Training Accuracy = 78.3133327960968 %
Validation Accuracy = 66.39999747276306 %
Validation loss = 22.42343521118164


In [52]:
confusion_matrix(y_test,y_pred)

array([[859,  91,  50],
       [137, 638, 225],
       [166, 339, 495]])